In [1]:
from numpy.lib.histograms import _histogram_dispatcher
import torch
from transformers import AutoModelForSequenceClassification,AutoTokenizer

from torchnlp.encoders import Encoder
from torchnlp.encoders.text import stack_and_pad_tensors
from torchnlp.encoders.text.text_encoder import TextEncoder
# from tokenizer import Tokenizer


import argparse
import os
from datetime import datetime
from pathlib import Path
import numpy as np

from bert_classifier import MimicBertModel, MimicDataset, MimicDataModule
import argparse
from datetime import datetime
import warnings

from data_utils import FewShotSampler, Mimic_ICD9_Processor, Mimic_ICD9_Triage_Processor, Mimic_Mortality_Processor



In [ ]:
model_dir= "./ckpts/icd9_triage/fewshot_32/emilyalsentzer/Bio_ClinicalBERT/version_28-03-2022--10-17-27/best-checkpoint.ckpt"
# hparams_file = "/home/niallt/NLP_DPhil/NLP_Mimic_only/clinical-longformer/experiments/emilyalsentzer/Bio_ClinicalBERT/version_20-09-2021--11-08-38/hparams.yaml"


model = MimicBertModel.load_from_checkpoint(model_dir)

print(model)

# # below works - but has to some funky shit

# hparams = bios.read(hparams_file)

# hparams = argparse.Namespace(**hparams)

# model = Classifier(hparams = hparams)

# checkpoint = torch.load(model_dir)

# model.load_state_dict(checkpoint['state_dict'])

# print(model)


In [14]:
import torch

x = torch.randn(3, 4)
x.shape
x

tensor([[ 1.0141,  0.2775,  0.1987,  0.2107],
        [-0.6086,  1.4512, -1.8213, -0.0802],
        [ 0.7498, -0.4312, -1.1538, -0.1927]])

In [15]:
y = torch.softmax(x, dim=1)
y

tensor([[0.4221, 0.2021, 0.1868, 0.1890],
        [0.0923, 0.7238, 0.0274, 0.1565],
        [0.5418, 0.1663, 0.0808, 0.2111]])

In [16]:
y = torch.softmax(x, dim=-1)
y

tensor([[0.4221, 0.2021, 0.1868, 0.1890],
        [0.0923, 0.7238, 0.0274, 0.1565],
        [0.5418, 0.1663, 0.0808, 0.2111]])

In [17]:
y = torch.softmax(x, dim=0)
y

tensor([[0.5089, 0.2116, 0.7188, 0.4140],
        [0.1004, 0.6842, 0.0953, 0.3095],
        [0.3907, 0.1042, 0.1859, 0.2766]])

In [4]:
model.hparams

"bert_hidden_dim":          768
"bert_model":               emilyalsentzer/Bio_ClinicalBERT
"ce_class_weights":         tensor([0.8571, 0.8571, 0.8571, 0.8571, 0.8571, 0.8571, 0.8571])
"class_labels":             ['AcuteMedicine', 'Cardiology', 'Gastroenterology', 'Neurology', 'Obstetrics', 'Oncology', 'Respiratory']
"classifier_hidden_dim":    768
"classifier_learning_rate": 1e-05
"dropout":                  0.1
"encoder_learning_rate":    1e-05
"n_training_steps":         840
"n_warmup_steps":           100
"nr_frozen_epochs":         0
"num_labels":               7
"optimizer":                adamw
"pretrained_dir":           None
"reinit_n_layers":          0
"weight_classes":           False

In [9]:
for param in model.classification_head.parameters():
    print(param)

Parameter containing:
tensor([[-0.0018, -0.0011,  0.0055,  ...,  0.0210,  0.0311, -0.0256],
        [ 0.0187,  0.0003, -0.0002,  ..., -0.0237, -0.0259, -0.0028],
        [-0.0312,  0.0288,  0.0182,  ..., -0.0226, -0.0232,  0.0299],
        ...,
        [ 0.0330,  0.0225,  0.0284,  ..., -0.0302, -0.0057, -0.0011],
        [ 0.0235,  0.0031, -0.0190,  ...,  0.0357, -0.0026, -0.0057],
        [ 0.0246, -0.0164,  0.0213,  ..., -0.0117,  0.0076,  0.0078]],
       requires_grad=True)
Parameter containing:
tensor([-3.3294e-02,  1.4990e-02, -2.9678e-02,  1.9077e-02,  2.9570e-02,
        -3.0759e-02, -2.9804e-02,  2.9669e-02,  3.2466e-02, -2.0307e-02,
        -3.0033e-02, -2.9377e-02,  1.0488e-02,  1.9232e-02, -3.2398e-02,
         9.1213e-04, -3.5529e-02,  2.3165e-02,  1.0563e-02,  1.7728e-02,
         6.8345e-03,  2.8228e-03,  9.8166e-03, -6.7819e-03, -3.5817e-02,
        -7.7066e-03,  3.0722e-02,  2.4454e-04, -3.0168e-02, -7.2411e-03,
        -3.3801e-02,  2.3773e-02,  2.9274e-02, -1.8287e-0

In [7]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params

108906247

In [24]:
model

MimicBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [2]:
# function for getting trainable parameters of a model

def get_n_trainable_params(model_dir):
    
    
    model = MimicBertModel.load_from_checkpoint(model_dir)
    
    # all trainable
    num_total_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # split into the plm and classisifcation head
    num_plm_trainable = sum(p.numel() for p in model.bert.parameters() if p.requires_grad)
    
    num_classifier_trainable = sum(p.numel() for p in model.classification_head.parameters() if p.requires_grad)
    
    # assert sum of the two = total
    assert num_plm_trainable+num_classifier_trainable== num_total_trainable
    
    print(f"Number of trainable parameters of PLM: {num_plm_trainable}\n")
    print('#'*50)
    print(f"Number of trainable parameters of classifier: {num_classifier_trainable}\n")
    print('#'*50)
    print(f"Total number of trainable parameters of whole model: {num_total_trainable}")
    
#     return model

In [3]:
# finetuned model
model_dir= "./ckpts/sensitivity/icd9_triage/fewshot_128_classhidden_2/frozen_plm/emilyalsentzer/Bio_ClinicalBERT/version_12-04-2022--09-46-40/best-checkpoint.ckpt"

get_n_trainable_params(model_dir)

2022-04-12 10:22:31.631 | WARNING  | bert_classifier:__init__:226 - Building model based on following architecture. emilyalsentzer/Bio_ClinicalBERT
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification m

Number of trainable parameters of PLM: 0

##################################################
Number of trainable parameters of classifier: 1559

##################################################
Total number of trainable parameters of whole model: 1559


In [6]:
3887/5

777.4